#<center> Draw a circle making the Lissajous figures </center>

In [2]:
import pandas as pd
import cv2
import numpy as np
import pandas as pd

In [102]:
# Function to generate Lissajous curve
def lissajous(t, A, B, a, b, delta):
    """
    Calculate the coordinates of a point on a Lissajous curve at given time 't'.

    Args:
        t (float): Time parameter.
        A (float): Amplitude of the sinusoidal motion along the x-axis.
        B (float): Amplitude of the sinusoidal motion along the y-axis.
        a (float): Frequency of the motion along the x-axis.
        b (float): Frequency of the motion along the y-axis.
        delta (float): Phase difference between the two sinusoidal motions.

    Returns:
        tuple: x and y coordinates of the point on the Lissajous curve at time 't'.

    Example:
        x, y = lissajous(0.5, 1.0, 1.5, 2.0, 3.0, np.pi/4)
    """
    x = A * np.sin(a * t)
    y = B * np.sin(b * t + delta)
    return x, y

def make_video(df,bck_img_path,
               videoPath='video.mp4',
               xShift = 0,
               yShift = 0,
               scaleFactor = 80,
               fps = 30,
               circle_radius = 8,
               circle_color = (0,0,255)):
    """
    Create a video from DataFrame points overlaid on a background image.

    Args:
        df (DataFrame): DataFrame containing 'x', 'y', and 't' columns for points.
        bck_img_path (str): Path to the background image.
        video_path (str): Path to save the output video. Default is 'video.mp4'.
        xShift (int): Horizontal shift for point positions. Default is 0.
        yShift (int): Vertical shift for point positions. Default is 0.
        scaleFactor (float): Scale factor of the trayectory.
        fps: Frames per second.
        circle_radius (int): Radius of the circle to draw for each point. Default is 8.
        circle_color (tuple): RGB color tuple for the circles. Default is (0,0,255) which is red.

    Returns:
        None: Saves the video to the specified path.

    Example:
        make_video(df, "background_image.jpg", "output_video.mp4", xShift=100, yShift=50)
    """
    img = cv2.imread(bck_img_path) # Background image. Try to be with this shape (648, 1152)
    down_width = int(img.shape[1])
    down_height = int(img.shape[0])


    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    out = cv2.VideoWriter(videoPath, fourcc, fps, (down_width, down_height))
    for i, row in df.iterrows():
        x, y, t = row['x'], row['y'], row['t']                           
        img_copy = np.copy(img)
        cv2.circle(img_copy, (int(x*scaleFactor+xShift), int((-y)*scaleFactor+yShift)), circle_radius, circle_color, -1) 
        down_points = (down_width, down_height)
        img2 = cv2.resize(img_copy, down_points, interpolation= cv2.INTER_LINEAR)
        out.write(img2)
    out.release()
    cv2.destroyAllWindows()

## Parameters

In [100]:
A = 2 # Vertical Amplitude
B = 2 # Horizontal Amplitud
a = 2 # Frequency of the motion along the x-axis.
b = 1 # Frequency of the motion along the y-axis.
delta = 0# Phase difference between the two sinusoidal motions.
num_points = 100 # amount of points
rounds = 5
fps = 30 # Frames per seconds
circle_radius = 8
circle_color = (0, 0, 255)
scale_factor = 80
x_shift = 576
y_shift = 324

img_path = 'background.png'
video_path = 'video.mp4'

## Define the trayectory of the point to view

In [84]:
df = pd.DataFrame()
df['t'] = np.linspace(0,vueltas*a*b*np.pi,num_points)
xx,yy = lissajous(df.t, A, B, a, b, delta)
df['x'] = xx
df['y'] = yy
df

,t,x,y
0,0.000000,0.000000e+00,0.000000e+00
1,0.317333,1.185816e+00,6.240669e-01
2,0.634665,1.909804e+00,1.185816e+00
3,0.951998,1.890002e+00,1.629152e+00
4,1.269330,1.134120e+00,1.909804e+00
...,...,...,...
95,30.146596,-1.134120e+00,-1.909804e+00
96,30.463929,-1.890002e+00,-1.629152e+00
97,30.781261,-1.909804e+00,-1.185816e+00
98,31.098594,-1.185816e+00,-6.240669e-01


## Make the video 

In [103]:
make_video(df, img_path,videoPath = video_path, xShift = x_shift, yShift = y_shift)